In [ ]:
using DataFrames, CSV, MLJ, VegaLite

In [ ]:
# Import CSV to dataframe, select some columns
lichen = CSV.read("C:/Users/julio/Downloads/predicted_biomass_Nov2021.csv", DataFrame; missingstring="") |>
x -> select(x, :biomass_log, :BASAL_AREA, :PL, :LON) 

In [ ]:
# schema 
schema(lichen)

In [ ]:
# Split predictors (X) and response variable (y)
y, X = unpack(lichen, ==(:biomass_log), _ -> true; rng=1010);

In [ ]:
# Search for models that match the data scitype and are pure Julia
for m in models(matching(X, y))
    if m.is_pure_julia == true
        println(rpad(m.name, 40), "($(m.package_name))")
    end
end

In [ ]:
# Load a RandomForestRegressor model using the DecisionTree package
model = @load RandomForestRegressor pkg="DecisionTree" add=true verbosity=0

In [ ]:
# Get model info
info(model)

In [ ]:
# Instantiate a random forest regressor model
model_rf = model(n_trees=50)

In [ ]:
# Create an MLJ machine (model + data)
mach_rf = machine(model_rf, X, y)

In [ ]:
# Split into train/test datasets
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=1010)

In [ ]:
# Fit the machine
fit!(mach_rf, rows=train)

In [ ]:
# Check trained parameters
fitted_params(mach_rf)

In [ ]:
# Make predictions
y_test_pred = predict(mach_rf, rows=test)

In [ ]:
# Get list of accuracy measures
# [m.name for m in measures() if m.target_scitype <: scitype(y)]
println([(m.name) for m in measures() if m.prediction_type == :deterministic])

In [ ]:
# Show some accuracy measures
@show rms(y_test_pred, y[test])
@show l1(y_test_pred, y[test]) |> mean
@show l2(y_test_pred, y[test]) |> mean
@show mav(y_test_pred, y[test])
@show rmsl(y_test_pred, y[test]);

In [ ]:
# Evaluate the model
evaluate(model_rf, X, y,
         resampling=CV(nfolds=3, shuffle=true),
         measure=rms, verbosity=1)

In [ ]:
# Evaluate the machine
# evaluate!(mach_rf,
#           resampling=Holdout(fraction_train=0.7, shuffle=true),
#           measure=rms, verbosity=1)
evaluate!(mach_rf,
          resampling=CV(nfolds=3, shuffle=true),
          measure=rms, verbosity=1)

In [ ]:
# Increase number of trees and re-evaluate the machine
model_rf.n_trees = 100
evaluate!(mach_rf,
          resampling=Holdout(fraction_train=0.7, shuffle=true),
          measure=rms, verbosity=1)